# Glove pretrained in the dataset

https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b

In [1]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [2]:
from methods.baseline import Baseline

Using TensorFlow backend.


In [3]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 2000

In [4]:
# Domain to use
DOMAIN = 'openoffice'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = 'baseline_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'baseline_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

In [5]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

In [6]:
baseline.load_ids(DIR)
len(baseline.bug_ids)

Reading bug ids


72234

In [7]:
def data_padding(data, max_seq_length):
    seq_lengths = [len(seq) for seq in data]
    seq_lengths.append(6)
    max_seq_length = min(max(seq_lengths), max_seq_length)
    padded_data = np.zeros(shape=[len(data), max_seq_length])
    for i, seq in enumerate(data):
        seq = seq[:max_seq_length]
        for j, token in enumerate(seq):
            padded_data[i, j] = int(token)
    return padded_data.astype(np.int)

In [8]:
import _pickle as pickle

def load_bugs(baseline):   
    removed = []
    baseline.corpus = []
    baseline.sentence_dict = {}
    baseline.bug_set = {}
    title_padding, desc_padding = [], []
    for bug_id in tqdm(baseline.bug_ids):
        try:
            bug = pickle.load(open(os.path.join(baseline.DIR, 'bugs', '{}.pkl'.format(bug_id)), 'rb'))
            title_padding.append(bug['title_word'])
            desc_padding.append(bug['description_word'])
            baseline.bug_set[bug_id] = bug
            #break
        except:
            removed.append(bug_id)
    
    if len(removed) > 0:
        for x in removed:
            baseline.bug_ids.remove(x)
        baseline.removed = removed
        print("{} were removed. To see the list call self.removed".format(len(removed)))

In [9]:
%%time

load_bugs(baseline)


CPU times: user 2.03 s, sys: 396 ms, total: 2.43 s
Wall time: 2.43 s


### Read the corpus from bugs

In [10]:
lines_title = [baseline.bug_set[bug_id]['title'] for bug_id in baseline.bug_ids]
lines_desc = [baseline.bug_set[bug_id]['description'] for bug_id in baseline.bug_ids]

lines = []
for title, desc in zip(lines_title, lines_desc):
    lines.append(title)
    lines.append(desc)
    
lines = [line.split(' ') for line in lines]

In [11]:
len(lines)

144468

## Glove embedding

In [12]:
%%time
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=3)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=300, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=100, no_threads=7, verbose=True)
glove.add_dictionary(corpus.dictionary)
#glove.save('glove.model')

Performing 100 training epochs with 7 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
CPU times: user 28min 23s, sys: 359 ms, total: 28min 23s
Wall ti

In [13]:
#glove_load = Glove.load('glove.model')

In [14]:
glove.word_vectors[glove.dictionary['eclipse']]

array([-3.01671178e-02,  7.96928956e-02, -4.89973891e-03,  8.24179463e-03,
        4.23435792e-02, -3.78799419e-03, -3.42506904e-02,  6.39346196e-03,
        1.16204392e-01,  1.82664258e-02,  3.52916974e-02,  7.26869927e-02,
       -4.71826341e-02,  3.81497582e-02,  4.69632607e-02, -8.18432812e-02,
       -3.63044191e-02,  2.08677544e-02, -2.48509894e-02,  4.07925195e-02,
       -1.64016014e-02,  1.99057800e-02, -7.32125915e-03, -2.88389479e-02,
       -5.47172078e-02, -1.69309184e-02, -9.36288072e-03, -4.86296529e-03,
        2.71142558e-02, -5.59808319e-03, -3.07014494e-03,  2.34035454e-02,
        3.40569184e-02,  2.29997874e-02, -2.37143885e-02,  7.81795026e-02,
       -7.31184359e-02, -3.05143408e-02,  1.70827616e-03, -4.15579461e-02,
        4.83921170e-02, -2.80120606e-02, -2.61907842e-02,  6.49267737e-03,
       -6.40725697e-02, -2.31659760e-02, -2.58620632e-02, -6.60426683e-02,
        1.08418261e-02,  6.23231217e-02,  2.03857202e-02, -6.28393616e-03,
       -2.04745524e-02, -

### Save dataset vocabulary embedding

In [15]:
import _pickle as pickle

In [16]:
vocab_embed = {}
for word in glove.dictionary:
    vocab_embed[word] = glove.word_vectors[glove.dictionary[word]].tolist()
    
with open(os.path.join(baseline.DIR, 'vocab_embed.pkl'), 'wb') as f:
      pickle.dump(vocab_embed, f)

In [17]:
vocab_embed['eclipse']

[-0.0301671178486638,
 0.07969289557631042,
 -0.0048997389135033475,
 0.008241794630271831,
 0.0423435791987109,
 -0.0037879941874158444,
 -0.0342506903662219,
 0.006393461962838885,
 0.11620439236625739,
 0.018266425818032606,
 0.03529169737064589,
 0.0726869927171718,
 -0.04718263405093645,
 0.03814975819757295,
 0.04696326069907318,
 -0.08184328118056311,
 -0.03630441906829318,
 0.020867754388676753,
 -0.024850989411993315,
 0.040792519519718747,
 -0.0164016014307359,
 0.019905780026029697,
 -0.007321259154192358,
 -0.028838947936175263,
 -0.054717207836744323,
 -0.01693091842435042,
 -0.009362880717395475,
 -0.004862965294570103,
 0.027114255845935582,
 -0.005598083185351236,
 -0.003070144940918858,
 0.02340354543498952,
 0.034056918407618736,
 0.02299978743661897,
 -0.023714388521969144,
 0.07817950256353656,
 -0.07311843594725051,
 -0.03051434083651547,
 0.0017082761583419148,
 -0.04155794612119744,
 0.048392116954753865,
 -0.028012060631495878,
 -0.02619078418360994,
 0.00649267